<a href="https://colab.research.google.com/github/37stu37/rnc2/blob/master/temporal_network_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import sys
import os
from pathlib import Path
import networkx as nx
from numba import jit
from dask.distributed import Client
pd.options.mode.chained_assignment = None  # default='warn'

client = Client(processes=False)
client

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/distributed/bokeh/core.py:57: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 99] Cannot assign requested address
  warnings.warn('\n' + msg)
/usr/local/lib/python3.6/dist-packages/distributed/deploy/local.py:197: UserWarning: 
Could not launch service 'bokeh' on port 8787. Got the following message:

[Errno 99] Cannot assign requested address
  self.scheduler.start(scheduler_address)


Client Scheduler: inproc://172.28.0.2/1433/1,Cluster Workers: 1 Cores: 2 Memory: 13.65 GB




---


**Tentative code for rnc2 temporal network**

@time

Conditions -> Boolean mask -> Contacts -> Edgelist


---



In [0]:
# import data
folder = Path('/content/drive/My Drive/04_Cloud/01_Work/Academia/01_Publications/00_Alex/005_RNC2')
edge_file = folder / 'data' / 'Copy of edge_data.parquet'
wind_file = folder / 'data' / 'Copy of GD_wind.csv'

wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

**Conditions**


---



**Main**


---



In [0]:
def wind_scenario(t, wind_data):
    if t == 0:
      i = np.random.randint(0, wind_data.values.shape[0])
      w = wind_data.values[i, 2]
      dist = wind_data.values[i, 1]
      b = wind_data.values[i, 3]
      bear_max = b + 45  # wind direction
      bear_min = b - 45
      if b == 360:
          bear_max = 45
      if b <= 0:  # should not be necessary
          bear_min = 0
      if b == 999:
          bear_max = 999
          bear_min = 0
      return bear_max, bear_min, dist # wind characteristics, bearing and distance
    else:
      return

def wind_mask(e, w_bearing_max, w_bearing_min, w_distance):
    mask1 = (e.bearing.values < w_bearing_max) & (e.bearing.values > w_bearing_min)
    mask2 = e.distance.values < w_distance
    mask = mask1 & mask2
    return mask

def ignition_mask(t, e, c):
    # initial ignition
    if t == 0:
      rng = np.random.uniform(0, 1, size=e.values.shape[0])
      mask = rng < e.IgnProb_bl.values
      return mask
    else:
    # propagation
      mask_previous_time = c.iloc[:, t-1] == 1
      previousTarget = e.target.iloc[mask_previous_time.values]
      mask = np.in1d(e.source.values, previousTarget.values)
      return mask

def not_burnt_mask(t, e, c):
    if t > 1:
      previouslyActivated = c.iloc[:,:t-1]
      previouslyActivated_sum = previouslyActivated.sum(axis=1)
      mask = np.where(previouslyActivated_sum> 0, 1, 0)
      mask = mask != 1
    else:
      mask = c.iloc[:,0].values
      mask = mask !=1
    return mask

In [0]:
n=10
for scenario in range(n):
    condition = True
    list_of_Activations = []
    time = 0 
    bear_max, bear_min, distance = wind_scenario(time, wind_data)
    CoTime = pd.DataFrame() # CoTime = pd.DataFrame(np.ones((len(edgelist), 1)))
    while condition = True:
        print("scenario : {} time : {}".format(scenario, time))
        # ignititon and propagation
        ignition_m = ignition_mask(time, edgelist, CoTime)
        if time == 0:
          CoTime.iloc[:,0] = mask
        # targets under the wind
        wind_m = wind_mask(edgelist, bear_max, bear_min, distance)
        # not already burnt
        not_burnt_m = not_burnt_mask(time, edgelist, CoTime)
        # resulting mask
        mask = (ignition_m & wind_m & not_burnt_m)
        # add mask to CoTime
        if time > 0:
          CoTime = pd.DataFrame(np.c_[CoTime, mask*1])
        # create Active edges
        ActiveEdges = edgelist[mask]
        if ActiveEdges is empty:

        print("number of active edges : {}".format(len(ActiveEdges)))
        # add scenario and time to active edges
        ActiveEdges["scenario"] = scenario
        ActiveEdges["time"] = time
        # append to list
        list_of_Activations.append(ActiveEdges)
        time += 1
    Activations = pd.concat(list_of_Activations)
    Activations.to_parquet(folder / 'output' / 'scenario{}_Activations.parquet'.format(scenario), engine='pyarrow')